In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
data = pd.read_csv('train.csv')
train = data.sample(frac=0.7, random_state=1)
test = data[data.index.isin(train.index)==False]
#train['Obs']=range(1,805)  # maybe change index from 1 to 804
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,RL,NaN,6853,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2009,WD,220000
791,160,FV,30.0,3180,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,151000
571,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,500,11,2009,WD,120500
224,20,RL,70.0,8050,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,127000
498,20,RL,NaN,9556,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,8,2007,WD,160000


In [2]:
rows, columns = train.shape
print(rows)
print(columns)

563
79


In [3]:
train.describe().round(2) # without object type

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,563.00,451.00,563.00,563.00,563.00,563.00,563.00,560.00,563.00,563.00,...,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00
mean,56.86,68.29,9800.64,5.97,5.63,1970.00,1982.61,103.04,439.10,54.17,...,90.55,41.44,25.37,1.73,14.31,1.02,21.76,6.06,2007.91,173867.17
std,41.65,22.60,5832.44,1.32,1.08,27.55,20.78,189.73,410.48,186.25,...,121.59,59.17,64.31,17.14,49.66,24.28,223.55,2.58,1.30,69561.87
min,20.00,21.00,1300.00,2.00,3.00,1880.00,1950.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,35000.00
25%,20.00,56.00,7490.00,5.00,5.00,1953.00,1963.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,129000.00
50%,50.00,69.00,9259.00,6.00,5.00,1970.00,1990.00,0.00,404.00,0.00,...,0.00,16.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,156000.00
75%,70.00,80.00,11366.50,7.00,6.00,1997.00,2002.00,148.00,695.50,0.00,...,166.00,62.00,0.00,0.00,0.00,0.00,0.00,7.00,2009.00,206500.00
max,190.00,155.00,70761.00,10.00,9.00,2008.00,2009.00,1224.00,1965.00,1526.00,...,646.00,365.00,364.00,219.00,287.00,576.00,4500.00,12.00,2010.00,538000.00


In [4]:
# variable preparation

predictors=list(train.columns[2:-1])
continuous_predictors=train.select_dtypes(['int64']).columns
columns_categorical=train.select_dtypes(['object']).columns


## Step 1: Categorial variables clean and discussion

In [5]:
train['MSZoning'].unique()

array(['RL', 'FV', 'RM', 'RH', 'C (all)', 'I (all)'], dtype=object)

In [6]:
# these variables are part of 'columns_categorical', 
categorical=['MSZoning', 'Street','LotShape', 'LandContour','Utilities','LotConfig', 'LandSlope'
           , 'Neighborhood', 'Condition1', 'Condition2' ,'BldgType','HouseStyle', 'RoofStyle', 'RoofMatl', 
             'Exterior1st','Exterior2nd','ExterQual', 'ExterCond', 'Foundation','Heating'
            , 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual','Functional'
            , 'PavedDrive','SaleType']


train['MSZoning']=train['MSZoning'].str.replace('(',"")
train['MSZoning']=train['MSZoning'].str.replace(')',"")
for item1 in categorical:
    train[item1]=train[item1].str.replace(' ',"")
    length=len(train[item1].unique())
    count=0
    while (length-1)>=count:
        for item2 in train[item1].unique():
            count1=str(count)
            train[item1]=train[item1].str.replace(item2,count1)
            count=int(count1)+1

In [7]:
#This is to replace NaN with empty string for now
train = train.replace(np.nan, '', regex=True)

#Use this to replace whatever first argument is with second argument
#train['Alley'] = train['Alley'].replace(0,'' , regex=True)


### Only 3 values with pool variables, we can just drop this

In [8]:
del train['PoolQC']
del train['PoolArea']
del test['PoolQC']
del test['PoolArea']

In [9]:
df = pd.DataFrame({'Fence':list(train['Fence'])})
pd.get_dummies(df).head()
#Here, the base case Fence means the house does not have a 

,Fence_,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


23 places have a feature out of 805 observations. Is it worthwhile to even include? Only 2% of observations have this

Therefore, I decided to remove it.

In [10]:
del train['MiscFeature']
del test['MiscFeature']

In [11]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,0,,6853,0,,0,0,0,0,...,25,0,0,0,,0,6,2009,0,220000
791,160,1,30,3180,0,Pave,1,0,0,0,...,166,0,0,0,,0,6,2006,0,151000
571,20,0,60,6960,0,,1,0,0,0,...,0,0,0,0,,500,11,2009,0,120500
224,20,0,70,8050,0,,1,0,0,0,...,64,0,0,0,,0,3,2007,0,127000
498,20,0,,9556,0,,0,0,0,1,...,21,0,0,0,,0,8,2007,0,160000


In [12]:
del train['Alley']
del test['Alley']

In [13]:
#Need to conver any string into integer for machine learning algorithm to work
combined_set = pd.concat([train, test], axis = 0)

In [14]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [15]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 8 to 802
Data columns (total 75 columns):
MSSubClass       804 non-null int64
MSZoning         804 non-null int8
LotFrontage      804 non-null int8
LotArea          804 non-null int64
Street           804 non-null int8
LotShape         804 non-null int8
LandContour      804 non-null int8
Utilities        804 non-null int8
LotConfig        804 non-null int8
LandSlope        804 non-null int8
Neighborhood     804 non-null int8
Condition1       804 non-null int8
Condition2       804 non-null int8
BldgType         804 non-null int8
HouseStyle       804 non-null int8
OverallQual      804 non-null int64
OverallCond      804 non-null int64
YearBuilt        804 non-null int64
YearRemod/Add    804 non-null int64
RoofStyle        804 non-null int8
RoofMatl         804 non-null int8
Exterior1st      804 non-null int8
Exterior2nd      804 non-null int8
MasVnrType       804 non-null int8
MasVnrArea       804 non-null int16
ExterQual    

In [16]:
final_train = combined_set[:train.shape[0]] # Up to the last initial training set row
final_test = combined_set[train.shape[0]:] # Past the last initial training set row

In [17]:
#This is the price variable we need to remove
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

In [19]:
import numpy
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Time for some Adaptive boosting with Decision Trees

In [20]:
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

### Testing to see if I can actually use Adaboost

In [21]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=9),random_state=0)
regr.fit(final_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50, random_state=0)

In [22]:
from sklearn.model_selection import cross_val_score
#This uses cross validation to evaluate the score
cross_val_score(regr, final_train, y_train, cv=10)

array([ 0.92690118,  0.81073481,  0.87656596,  0.91526276,  0.88009621,
        0.8243043 ,  0.91016168,  0.85105471,  0.92049621,  0.87521091])

# Bringing it all together to find optimal level of nodes to use!

In [ ]:
values = range(1,20)
maximum = 0
optimal_node = 0
for i in values:
    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=i),random_state=0)
    print("Cross val score for {} nodes".format(i))
    print(cross_val_score(regr, final_train, y_train, cv=10))
    tot = 0
    #Check to see if this layer is better since highest cross_val_score
    for x in g:
        tot = tot+x
    if tot>maximum:
        optimal_node = i
       

In [23]:
from sklearn.metrics import r2_score
print("Optimal nodes is {}".format(19))
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19),random_state=0)
print("Cross val score for {} nodes".format(19))
print(cross_val_score(regr, final_train, y_train, cv=10))

regr.fit(final_train, y_train)
y_1 = regr.predict(final_test)
accuracy = r2_score(y_test, y_1)
print("Cross-Predicted Accuracy: {}".format(accuracy))

Optimal nodes is 19
Cross val score for 19 nodes
[ 0.92433361  0.81603069  0.88490677  0.90816879  0.88445049  0.81368237
  0.90009285  0.87982209  0.90687126  0.88195048]
Cross-Predicted Accuracy: 0.857095403248


In [24]:
from QBUS2820 import rmse_jack, r2_jack 
#Jackknife method to be able to report the standard errors for the test results

columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Results']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, y_1)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, y_1))
results.round(3)

,Test RMSE,SE,Test R2,SE
Results,26844.18,4555.389,0.857,0.031


# Here we create an esemble by using Lasso in addition to it

In [25]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)

In [26]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [27]:
#Combining the 2 predictions
pred_combined = (pred_L+pred_T)/2

In [28]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
Lasso,32733.989,4229.731,0.788,0.035
Adaboost Tree,26844.180,4555.389,0.857,0.031
Lasso + Tree,25356.139,4792.178,0.872,0.031


# Worked! Let's try combining ridge and Enet too

In [29]:
#Elastic Net
from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(final_train, np.ravel(y_train))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(y_train))

ElasticNet(alpha=166961.3766010862, copy_X=True, fit_intercept=True,
      l1_ratio=0.98999999999999999, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [30]:
#Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(final_train, np.ravel(y_train))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(y_train))

Ridge(alpha=15.572257753236348, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [31]:
pred_R = ridge.predict(final_test)
pred_L = lasso.predict(final_test)
pred_E = enet.predict(final_test)

In [32]:
#Combining the all predictions
pred_combined = (pred_L+pred_T)/2
pred_combined2 = (pred_R+pred_T)/2
pred_combined3 = (pred_E+pred_T)/2
pred_combined4 = (pred_L+ + pred_R + pred_E + pred_T)/4

In [33]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree', 'Ridge', 'Enet', 'Tree + Ridge', 'Tree + Enet', 'Tree + All 3']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))
results.iloc[3,0], results.iloc[3,1] = rmse_jack(y_test, pred_R)
results.iloc[3,2], results.iloc[3,3] = (r2_jack(y_test, pred_R))
results.iloc[4,0], results.iloc[4,1] = rmse_jack(y_test, pred_E)
results.iloc[4,2], results.iloc[4,3] = (r2_jack(y_test, pred_E))
results.iloc[5,0], results.iloc[5,1] = rmse_jack(y_test, pred_combined2)
results.iloc[5,2], results.iloc[5,3] = (r2_jack(y_test, pred_combined2))
results.iloc[6,0], results.iloc[6,1] = rmse_jack(y_test, pred_combined3)
results.iloc[6,2], results.iloc[6,3] = (r2_jack(y_test, pred_combined3))
results.iloc[7,0], results.iloc[7,1] = rmse_jack(y_test, pred_combined4)
results.iloc[7,2], results.iloc[7,3] = (r2_jack(y_test, pred_combined4))
results.round(3)

,Test RMSE,SE,Test R2,SE
Lasso,32733.989,4229.731,0.788,0.035
Adaboost Tree,26844.180,4555.389,0.857,0.031
Lasso + Tree,25356.139,4792.178,0.872,0.031
Ridge,50372.276,2508.451,0.497,0.065
Enet,34323.273,3971.281,0.766,0.035
Tree + Ridge,33309.396,3431.868,0.780,0.024
Tree + Enet,25904.156,4639.640,0.867,0.030
Tree + All 3,28395.976,4231.660,0.840,0.027


# From this, it appears that Lasso + Tree is the best Model

# Let's try Tree + PCR

In [81]:
from QBUS2820 import pcrCV
pcr=pcrCV(final_train, y_train)

In [82]:
pred_pcr = pcr.predict(final_test)

In [83]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [84]:
pred_combined = (pred_pcr+pred_T)/2

In [38]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['PCR', 'Adaboost Tree', 'PCR + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_pcr)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_pcr))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
PCR,33457.064,3182.188,0.778,0.031
Adaboost Tree,26844.180,4555.389,0.857,0.031
PCR + Tree,26370.503,3857.765,0.862,0.023


In [39]:
#Adding lasso back now
pred_L = lasso.predict(final_test)

In [40]:
pred_combined = (pred_pcr+pred_T+pred_L)/3

In [41]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = [ 'LASSO + PCR + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_combined)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
LASSO + PCR + Tree,25590.174,4302.072,0.87,0.026


# Lasso + Tree seems to do better without dimension reduction

# Trying out model stacking now where we combine both lasso and tree to be predictors in an OLS regression

In [85]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [86]:
#Adding lasso back now
pred_L = lasso.predict(final_test)

In [87]:
print(len(final_train))
print(len(y_train))
print(len(final_test))
print(len(y_test))

563
563
241
241


In [88]:
import numpy as np
import pandas as pd

stacked_lasso_tree = pd.DataFrame({'Lasso':pred_L, 'Tree':pred_T, 'Price':y_test})
#Created new dataframe with both predictions as values

In [89]:
import statsmodels.formula.api as sm
result = sm.ols(formula="Price ~ Lasso + Tree", data=stacked_lasso_tree).fit()

In [90]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     877.9
Date:                Thu, 31 Aug 2017   Prob (F-statistic):          1.42e-110
Time:                        10:51:12   Log-Likelihood:                -2778.0
No. Observations:                 241   AIC:                             5562.
Df Residuals:                     238   BIC:                             5572.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3061.1324   4856.230     -0.630      0.529   -1.26e+04    6505.551
Lasso          0.3371      0.056      6.071      0.000       0.228       0.447
Tree           0.6946      0.051     13.574      0.000       0.594       0.795
==============================================================================
Omnibus:                      198.899   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7046.704
Skew:                           2.841   Prob(JB):                         0.00
Kurtosis:                      28.874   Cond. No.                     8.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [97]:
predictions = result.predict(exog=stacked_lasso_tree) 

In [98]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = [ 'Stacked Model with Tree and LASSO']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, predictions)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, predictions))

results.round(3)

,Test RMSE,SE,Test R2,SE
Stacked Model with Tree and LASSO,24534.681,4577.021,0.881,0.029
